In [12]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd

In [13]:
URL = 'https://snap.fan/bundles/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [51]:
bundles = []

In [52]:
# 2025-04-30

# Trova tutte le bundle card
bundle_cards = soup.find_all('div', class_='bundle-card')

# Estrai i campi desiderati
for card in bundle_cards:
    # print(card)
    name = card.find('div', class_='bundle-card__name').text
    # print(name)
    card_location = card.find('div', {'data-card-def-tooltip-app': True})
    if card_location is None:
        card_name = ''
    else:
        card_name = card_location['data-card-def-tooltip-app']
    # print(card)
    date = card.find('span', class_='banner-date__dates').get_text(strip=True)
    # print(date)
    cost = card.find('div', class_='bundle-card__cost').get_text(strip=True)
    # print(cost)
    
    cost_regex = '(\$\d*\.\d*|\d*)(Bundle Value)(\d*)%(Currency Value)(\d*)%'
    cost_regex_find = re.findall(cost_regex, cost)
    
    if not cost_regex_find:
        continue
    
    if cost_regex_find[0][0].startswith('$'):
        gold_cost = 0
        dollar_cost = float(cost_regex_find[0][0][1:])
    else:
        gold_cost = int(cost_regex_find[0][0])
        dollar_cost = 0
    bundle_value = int(cost_regex_find[0][2])
    currency_value = int(cost_regex_find[0][4])
    
    # Trova tutti gli item all'interno della bundle card
    items = card.find_all('div', class_='bundle-card__item')
    currencies = []
    for item in items:
        item_text = item.get_text(strip=True)
        if 'Tokens' in item_text:
            currencies.append({'tokens': int(item_text.split(' ')[0])})
        elif 'Credits' in item_text:
            currencies.append({'credits': int(item_text.split(' ')[0])})
        elif 'Gold' in item_text:
            currencies.append({'gold': int(item_text.split(' ')[0])})
    
    if currencies:
        bundle = {
            'name': name,
            'date': date,
            'dollar_cost': dollar_cost,
            'gold_cost': gold_cost,
            'bundle_value': bundle_value,
            'currency_value': currency_value,
            'tokens': 0,
            'credits': 0,
            'gold': 0,
            'card': card_name,
        }
        for curr in currencies:
            bundle = {**bundle, **curr}
        bundle['tokens_to_gold'] = round(bundle['tokens'] * 1.25)
        bundle['credits_to_gold'] = round(bundle['credits'] * 0.8)
        bundle['gold_total'] = bundle['gold'] + bundle['tokens_to_gold'] + bundle['credits_to_gold']
        bundle['estimated_dollar_cost'] = round(bundle['gold_total'] * 10 / 8 / 100)
        bundle['dollar_gain'] = 0
        bundle['gold_gain'] = 0
        if bundle['dollar_cost']:
            bundle['dollar_gain'] = round(bundle['estimated_dollar_cost'] / bundle['dollar_cost'] * 100)
        if bundle['gold_cost']:
            bundle['gold_gain'] = round(bundle['gold_total'] / bundle['gold_cost'] * 100)
        
        bundles.append(bundle)

In [53]:
df = (
    pd.DataFrame(bundles)
)

df.to_clipboard()

In [54]:
(
    df
    .sort_values('dollar_gain', ascending=False).head(10)
    [['name', 'date', 'card', 'dollar_cost', 'bundle_value', 'currency_value', 'tokens', 'credits']]
)

,name,date,card,dollar_cost,bundle_value,currency_value,tokens,credits
0,Webshop Conversion 01,Oct 17 - Dec 30,IronMan,2.99,915,502,0,1500
2,Apr 25 Iro,Apr 25 - May 2,IronFist,49.99,221,195,0,6000
21,Chicken Before the Egg,May 11 - May 16,ThePhoenixForce,14.99,587,192,0,1000
60,I'm Merry Poppins Y'all,Jun 15 - Jun 20,Yondu,9.99,386,188,0,0
99,Jul 25 Thi,Jul 25 - Jul 28,TheThing,9.99,311,188,0,0
57,Collector's Bundle,Jun 11 - Jun 16,,24.99,189,188,3000,0
106,Jul 25 Webshop Recap 01,Jul 31 - Aug 7,,14.99,187,183,0,2000
10,Apr 25 Webshop Recap 02,May 1 - May 8,Scorpion,14.99,278,183,0,500
107,Jul 25 Webshop Recap 02,Jul 31 - Aug 7,,14.99,187,183,0,500
9,Apr 25 Webshop Recap 01,May 1 - May 8,Electro,14.99,278,183,0,2000


In [60]:
(
    df
    .query('dollar_cost < 99')
    .sort_values('dollar_gain', ascending=False)
    [['name', 'date', 'dollar_cost', 'card', 'bundle_value', 'currency_value', 'tokens', 'credits']]
    .head(20)
)

,name,date,dollar_cost,card,bundle_value,currency_value,tokens,credits
0,Webshop Conversion 01,Oct 17 - Dec 30,2.99,IronMan,915,502,0,1500
2,Apr 25 Iro,Apr 25 - May 2,49.99,IronFist,221,195,0,6000
21,Chicken Before the Egg,May 11 - May 16,14.99,ThePhoenixForce,587,192,0,1000
99,Jul 25 Thi,Jul 25 - Jul 28,9.99,TheThing,311,188,0,0
60,I'm Merry Poppins Y'all,Jun 15 - Jun 20,9.99,Yondu,386,188,0,0
57,Collector's Bundle,Jun 11 - Jun 16,24.99,,189,188,3000,0
10,Apr 25 Webshop Recap 02,May 1 - May 8,14.99,Scorpion,278,183,0,500
107,Jul 25 Webshop Recap 02,Jul 31 - Aug 7,14.99,,187,183,0,500
106,Jul 25 Webshop Recap 01,Jul 31 - Aug 7,14.99,,187,183,0,2000
9,Apr 25 Webshop Recap 01,May 1 - May 8,14.99,Electro,278,183,0,2000


In [61]:
(
    df
    .query('dollar_cost < 99')
    .sort_values('bundle_value', ascending=False)
    [['name', 'date', 'dollar_cost', 'card', 'bundle_value', 'currency_value', 'tokens', 'credits']]
    .head(20)
)

,name,date,dollar_cost,card,bundle_value,currency_value,tokens,credits
0,Webshop Conversion 01,Oct 17 - Dec 30,2.99,IronMan,915,502,0,1500
21,Chicken Before the Egg,May 11 - May 16,14.99,ThePhoenixForce,587,192,0,1000
8,Stealth Selfie,May 1 - May 6,24.99,WhiteWidow,585,160,0,4000
32,Otaku Outlaw,May 20 - May 27,24.99,Gwenpool,581,156,2500,0
85,Monkey Business,Jul 10 - Jul 15,14.99,HitMonkey,562,167,0,1000
72,Beware The Dark Web,Jun 26 - Jul 1,29.99,SymbioteSpiderMan,508,154,0,1500
65,Shi'ar Shadows,Jun 20 - Jun 25,24.99,Gladiator,397,160,0,2500
88,The Jade Web Weaver,Jul 15 - Jul 22,49.99,Arana,393,165,4000,2000
7,Lady Lash,Apr 30 - May 5,24.99,Valentina,393,156,2500,0
55,Spellborn Heir,Jun 10 - Jun 17,49.99,Wiccan,393,165,0,2000


In [62]:
(
    df
    .query('gold_cost > 1000')
    .sort_values('gold_gain', ascending=False)
    [['name', 'date', 'gold_cost', 'card', 'bundle_value', 'currency_value', 'tokens', 'credits']]
    .reset_index(drop=True)
)

,name,date,gold_cost,card,bundle_value,currency_value,tokens,credits
0,Ethereal Architect,May 8 - May 13,5000,Sersi,294,123,3000,3000
1,Sir Hydration Robert,Jun 12 - Jun 17,5000,HydraBob,294,123,3000,3000
2,Cold Embrace,Jun 19 - Jun 24,5000,Death,143,123,3000,3000
3,Frostfang Behemoth,Jul 9 - Jul 14,5000,Sasquatch,216,120,0,7500
4,Twilight Prince,Jul 31 - Aug 5,5000,Malekith,286,115,3000,2500
5,Fairy Dust Fighter,May 27 - Jun 3,5000,Pixie,220,112,4500,0
6,Escape Velocity,Jul 3 - Jul 8,2500,Quinjet,152,112,0,3500
7,Talon Tyrant,Jul 16 - Jul 21,2500,Vulture,147,107,1500,1000
8,Poised for Piracy,Jun 5 - Jun 10,3500,Nebula,136,107,3000,0
9,Femme Fatale,May 16 - May 21,5000,Diamondback,275,104,0,6500
